# Graph Convolution Network

In [ ]:
from __future__ import division

The statement "from future import division" is used in Python 2.x to change the division behavior to match that of Python 3.x. In Python 2.x, the division operator "/" performs floor division when used with two integers, meaning that the result is rounded down to the nearest integer. This behavior can sometimes lead to unexpected results, especially when working with floating-point numbers.

By including "from future import division" at the beginning of a Python script, it changes the behavior of the division operator so that it always performs true division, even when used with two integers. True division means that the result of the division operation is a floating-point number, rather than an integer.

In Python 3.x, true division is the default behavior, so this statement is not needed. However, in Python 2.x, it is necessary if you want to ensure that your code behaves consistently with Python 3.x.

In [ ]:
from __future__ import print_function

By including "from future import print_function" at the beginning of a Python script, it enables the use of the print() function, which allows for more flexible printing of values. The print() function requires parentheses around the values being printed and can accept multiple arguments, which are then printed with a space separator by default.

It is important to note that once "from future import print_function" is included in a script, all subsequent print statements must use the print() function with parentheses, rather than the print statement without parentheses. This is because the statement changes the behavior of the print statement to match that of the print() function in Python 3.x.

In [ ]:
import time
import argparse


In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
from utils import load_data, accuracy

The "load_data" function loads the Cora citation network dataset and returns it as a collection of NumPy arrays and PyTorch tensors.

Here's how the function works:

- The function first loads the node features and labels from the file "cora.content" using the NumPy function np.genfromtxt(). This file is assumed to be located in the "../data/cora/" directory, but the path can be customized using the "path" argument. The resulting array "idx_features_labels" has shape (n_nodes, n_features + 1), where n_nodes is the number of nodes in the graph and n_features is the number of features for each node.
- The function extracts the features from the "idx_features_labels" array by selecting all columns except the first and last. The resulting array is converted to a sparse CSR matrix using the function sp.csr_matrix().
- The function extracts the labels from the "idx_features_labels" array by selecting the last column and one-hot encoding the labels using the encode_onehot() function defined elsewhere. The resulting array of one-hot encoded labels has shape (n_nodes, n_classes), where n_classes is the number of unique labels in the dataset.
- The function loads the edge list from the file "cora.cites" using the NumPy function np.genfromtxt(). This file contains pairs of node IDs representing edges in the graph. The function maps these node IDs to the corresponding row indices in the features and labels arrays using a dictionary called "idx_map", and constructs a sparse COO matrix called "adj" to represent the adjacency matrix of the graph.
- The function then applies some preprocessing steps to the features and adjacency matrix. The features are normalized using the normalize() function, which subtracts the mean and divides by the standard deviation of each feature. The adjacency matrix is normalized using the normalize() function and transformed into a sparse PyTorch tensor using the function sparse_mx_to_torch_sparse_tensor().
- Finally, the function creates PyTorch tensors for the training, validation, and test indices, and returns all of the preprocessed data as a tuple of PyTorch tensors.

Overall, this function loads and preprocesses the Cora citation network dataset for use in a graph neural network.

The "accuracy" function computes the classification accuracy of a set of predictions given the true labels. Here's how the function works:

- The function takes as input two PyTorch tensors: "output" and "labels". "output" is assumed to be a tensor of shape (n_nodes, n_classes) containing the predicted class probabilities for each node in the graph. "labels" is a tensor of shape (n_nodes,) containing the true class labels for each node.
- The function first computes the predicted class for each node by taking the index of the maximum value along the second dimension (i.e., the class dimension) of the "output" tensor. This is done using the PyTorch function max(1)[1], which returns a tensor of shape (n_nodes,) containing the predicted class indices.
- The function then converts the predicted class indices to the same data type as the "labels" tensor using the function type_as(labels).
- The function computes a boolean tensor called "correct" by checking whether each predicted class is equal to the true class label. This is done using the PyTorch function eq(), which returns a tensor of shape (n_nodes,) containing 1 for correct predictions and 0 for incorrect predictions.
- The function computes the total number of correct predictions by summing the "correct" tensor, and converts the result to a double precision floating-point number using the function double().
- Finally, the function returns the classification accuracy as the ratio of correct predictions to the total number of nodes in the graph, computed as correct / len(labels).

Overall, this function provides a convenient way to evaluate the classification accuracy of a set of predictions on a graph classification task, using PyTorch tensors.

In [ ]:
import models

In [ ]:
from models import GCN


#### Training Settings

In [ ]:
parser = argparse.ArgumentParser()


In [ ]:
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')

_StoreTrueAction(option_strings=['--no-cuda'], dest='no_cuda', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Disables CUDA training.', metavar=None)

In [ ]:
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')

_StoreTrueAction(option_strings=['--fastmode'], dest='fastmode', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Validate during training pass.', metavar=None)

In [ ]:
parser.add_argument('--seed', type=int, default=42, help='Random seed.')

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=42, type=<class 'int'>, choices=None, required=False, help='Random seed.', metavar=None)

In [ ]:
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')

_StoreAction(option_strings=['--epochs'], dest='epochs', nargs=None, const=None, default=200, type=<class 'int'>, choices=None, required=False, help='Number of epochs to train.', metavar=None)

In [ ]:
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')

_StoreAction(option_strings=['--lr'], dest='lr', nargs=None, const=None, default=0.01, type=<class 'float'>, choices=None, required=False, help='Initial learning rate.', metavar=None)

In [ ]:
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')

_StoreAction(option_strings=['--weight_decay'], dest='weight_decay', nargs=None, const=None, default=0.0005, type=<class 'float'>, choices=None, required=False, help='Weight decay (L2 loss on parameters).', metavar=None)

In [ ]:
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')

_StoreAction(option_strings=['--hidden'], dest='hidden', nargs=None, const=None, default=16, type=<class 'int'>, choices=None, required=False, help='Number of hidden units.', metavar=None)

In [ ]:
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

_StoreAction(option_strings=['--dropout'], dest='dropout', nargs=None, const=None, default=0.5, type=<class 'float'>, choices=None, required=False, help='Dropout rate (1 - keep probability).', metavar=None)

In [ ]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--no-cuda] [--fastmode] [--seed SEED]
                             [--epochs EPOCHS] [--lr LR]
                             [--weight_decay WEIGHT_DECAY] [--hidden HIDDEN]
                             [--dropout DROPOUT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/vr-lab/.local/share/jupyter/runtime/kernel-fd7c422d-ea24-4588-93ad-2133635fb8fc.json


SystemExit: 2